# __Sistema de Recomendacion Predictivo de Productos__

## __Resumen y Contexto Comercial__:

`Cervecería y Maltería Quilmes` (`CMQ`) tiene una aplicación B2B llamada `BEES`, que permite a empresas realizar pedidos de productos. Tras realizar encuestas de satisfacción del cliente, se identificó que uno de los puntos clave para mejorar la experiencia de usuario es la creación de un nuevo módulo que `ayude a los clientes a encontrar los productos que son más propensos a comprar en su próximo pedido`. Esto busca `reducir el tiempo dedicado a completar cada compra`, mejorando la experiencia general.

El equipo de ciencia de datos de `CMQ` ha sido encargado de desarrollar un modelo de Machine Learning que prediga qué productos comprará cada cliente en su siguiente pedido. Este modelo será integrado en la aplicación `BEES`, generando recomendaciones personalizadas de productos. El equipo de producto utilizará el output del modelo para diseñar y desplegar el nuevo módulo dentro de la app, mientras que el equipo de ingeniería se encargará de la integración del modelo en la arquitectura de datos mediante pipelines.

El objetivo principal es que el modelo prediga las compras a nivel individual de cada cliente de manera diaria, basado en patrones de compra anteriores y atributos asociados a cada cliente y producto.

## __Características de los Datos__

#### **Atributos**: Atributos asociados a cada cliente, proporcionando un perfil detallado de sus comportamientos y características.

- **`Unnamed: 0`**: Índice generado automáticamente, no relevante para el análisis.

- **`POC`**: Código de identificación único del cliente, equivalente a `ACCOUNT_ID` en la tabla de transacciones.

- **`BussinessSegment`**: Clasificación del cliente según su nivel de uso de la aplicación BEES. Puede tomar los valores `PowerUsage`, `HighUsage`, `MediumUsage`, y `MinimalUsage`.

- **`totalVolumen`**: Volumen total en hectolitros adquirido por el cliente durante el período de análisis (junio-agosto).

- **`SkuDistintosPromediosXOrden`**: Promedio de productos distintos comprados por el cliente en cada orden durante el período de análisis.

- **`SkuDistintosToTales`**: Total de productos distintos adquiridos por el cliente en el período.

- **`concentracion`**: Estimación de la concentración de negocios diferentes a 150 metros del punto de venta. Puede tener los valores `Alto`, `Medio`, `Bajo`, o `S/D` (Sin Datos).

- **`nse`**: Nivel socioeconómico de los habitantes cercanos al punto de venta. Puede ser `Alto`, `Medio`, `Bajo`, o `S/D` (Sin Datos).

- **`segmentoUnico`**: Clasificación del cliente basada en su capacidad de compra. Puede tomar los valores `Inactivos`, `Masivos`, `Potenciales`, `Activos`.

- **`canal`**: Canal de marketing al que pertenece el cliente, con valores como `COMIDA`, `Tradicional`, `BEBIDA`, `Mayorista`, `Kioscos/Maxikioscos`, entre otros.


#### **Transacciones**: Contiene información de las transacciones realizadas por los clientes a través de la aplicación BEES de CMQ. Cada fila representa una compra específica realizada por un cliente.

- **`Unnamed: 0`**: Índice generado automáticamente, no relevante para el análisis.

- **`ACCOUNT_ID`**: Código de identificación único para cada cliente.

- **`SKU_ID`**: Código de identificación único de cada producto vendido.

- **`INVOICE_DATE`**: Fecha en la que se realizó la transacción, en formato numérico `yyyyMMdd`.

- **`ORDER_ID`**: Código identificador del pedido.

- **`ITEMS_PHYS_CASES`**: Número de bultos físicos comprados de un producto específico en la transacción.


Esta descripción proporciona una visión clara de los datos disponibles en ambas tablas, lo que permitirá realizar análisis exploratorios y aplicar técnicas de machine learning para las recomendaciones de productos.

## __Carga de Datos__

Leer los archivos `atributos.csv` y `transactions.csv`

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

In [2]:
# Cargar los datos
clients = pd.read_csv('atributos.csv')
transactions = pd.read_csv('transacciones.csv')

* #### __`Clientes`__

In [ ]:
# Mostrar los primeros registros
clients.head()

In [ ]:
# Mostrar la estructura de los datos
clients.shape

In [ ]:
# Información del DataFrame clientes
clients.info()

Observamos que la base de datos `clientes` esta compuesta por 4400 registros y 10 columnas. Por otro lado, renombraremos las columnas para mayor comprensión, considerando buenas prácticas de código.

In [6]:
# Renombrar columnas
new_cols = ['UNNAMED: 0', 'ACCOUNT_ID', 'BUSINESS_SEGMENT', 'TOTAL_VOLUME', 'DIFF_SKU_BY_ORDER',
            'TOTAL_DIFF_SKU', 'CONCENTRATION', 'NSE', 'UNIQUE_SEGMENT', 'CHANNEL']

clients.columns = new_cols

In [ ]:
# Número de clientes únicos
clients['ACCOUNT_ID'].nunique()

In [ ]:
# Verificamos la existencia de duplicados o datos nulos
clients_duplicated = clients.duplicated().sum()
clients_null = clients.isnull().sum()

print(f"Clientes Duplicados: {clients_duplicated}")
print()
print(f"Clientes Nulos:\n\n{clients_null}")

In [ ]:
# Porcentaje de valores nulos por columna
clients_null_percent = clients_null / len(clients) * 100
print(f"Porcentaje de valores nulos por columna:\n\n{clients_null_percent}")

Observamos que la columna `UNIQUE_SEGMENT` presenta 75 valores nulos y la columna `CHANNEL` 14. Por lo que investigaremos aquellas filas.

In [ ]:
# Clientes con valores nulos
clients[clients['UNIQUE_SEGMENT'].isna()]

In [ ]:
# Conteo de valores para la columna CHANNEL
clients[clients['UNIQUE_SEGMENT'].isna()]['CHANNEL'].value_counts(dropna=False)

No hay filas duplicadas en el dataset `clientes`, lo cual es positivo ya que no será necesario realizar ningún tratamiento adicional para eliminarlos.

Clientes con valores nulos:

* `UNIQUE_SEGMENT`: Hay 75 registros con valores faltantes.

* `CHANNEL`: Hay 14 registros con valores faltantes, pertenecientes a `UNIQUE_SEGMENT`.

En la proxima sección tendremos que decidir si eliminar aquellos valores faltantes, imputarlos o crear una nueva característica llamada `Desconocido`.

In [ ]:
# Conteo de valores para variables categóricas
print(clients['BUSINESS_SEGMENT'].value_counts())
print()
print(clients['CONCENTRATION'].value_counts())
print()
print(clients['NSE'].value_counts())
print()
print(clients['UNIQUE_SEGMENT'].value_counts(dropna=False))

In [ ]:
# Conteo de valores para la columna CHANNEL
clients['CHANNEL'].value_counts(dropna=False)

In [ ]:
# Descripción estadística de clientes
clients.describe()

En resumen, hay una alta variabilidad en el comportamiento de los clientes en términos de volumen y variedad de productos comprados.

* #### `Transacciones`

In [ ]:
# Mostrar los primero registros para transacciones
transactions.head()

In [ ]:
# Cantidad de filas y columnas
transactions.shape

In [ ]:
# Información del dataset
transactions.info()

In [ ]:
# Verificar duplicados y valores nulos
print(f"Transacciones Duplicadas: {transactions.duplicated().sum()}")
print()
print(f"Transacciones Nulas:\n\n{transactions.isna().sum()}")

In [ ]:
# Cantidad de ordenes, clientes, productos y fechas unicas
print(f"Ordenes unicas: {transactions['ORDER_ID'].nunique()}")
print(f"Clientes unicos: {transactions['ACCOUNT_ID'].nunique()}")
print(f"Productos unicos: {transactions['SKU_ID'].nunique()}")
print(f"Fechas de compra unicas: {transactions['INVOICE_DATE'].nunique()}")

`Tamaño del Dataset`: El conjunto de datos tiene 280,828 filas y 6 columnas, lo que muestra un registro detallado de las transacciones de la compañía.

`Información de Columnas`: No hay valores nulos ni duplicados en ninguna de las columnas. Las columnas contienen principalmente datos numéricos, excepto por ORDER_ID, que es de tipo object, lo que indica que se trata de un identificador alfanumérico.

`Unicidad`:

* __Ordenes únicas__: Hay 45,547 órdenes de compra distintas.

* __Clientes únicos__: El conjunto de datos incluye 4,535 clientes diferentes.

* __Productos únicos__: Se compraron 530 productos distintos.

* __Fechas de compra únicas__: Las compras se distribuyeron a lo largo de 77 fechas diferentes, lo que indica que los datos cubren un período considerable de tiempo.

In [ ]:
# Descripción estadística de las transacciones
transactions.describe()

La mayoría de los clientes realiza compras pequeñas, con un promedio de casi 4 productos por transacción, aunque hay transacciones mucho mayores que llegan hasta 2,000 productos. Las fechas de las transacciones son consistentes en 2022, y existe una gran dispersión en los tipos de productos comprados. La distribución de clientes y productos sugiere que hay comportamientos de compra variados que podrían ser útiles para los modelos de recomendación.

## __Transformacion de Datos__

* #### __`Clientes`__

La columna `UNNAMED: 0` no contiene información relevante por lo que la eliminaremos del Dataset.

In [21]:
# Eliminar la columna 'UNNAMED: 0'
clients.drop('UNNAMED: 0', axis=1, inplace=True)

Para evitar introducir sesgos en los datos, no realizaremos imputaciones ni crearemos una nueva característica que podría aumentar la cardinalidad del problema. Dado que los valores nulos en las columnas `UNIQUE_SEGMENT` y `CHANNEL` representan menos del 2% del total, eliminaremos esas filas para conservar la integridad de la base de datos.

In [ ]:
# Eliminar las filas con valores nulos
clients.dropna(inplace=True)

# Verificar que no hay valores nulos
clients.isna().sum()

* #### __`Transacciones`__

La columna `Unnamed: 0` no contiene información relevante por lo que la eliminaremos del Dataset.

In [23]:
# Eliminar la columna 'Unnamed: 0'
transactions.drop('Unnamed: 0', axis=1, inplace=True)

La columna `INVOIDE_DATE` esta en formato `int64` por lo que la transformaremos a formato `datetime` para crear carácteristicas temporales que pueden ser útiles para el modelo de machine learning.

In [24]:
# Convertir INVOICE_DATE a datetime
transactions['INVOICE_DATE'] = pd.to_datetime(transactions['INVOICE_DATE'], format='%Y%m%d')

## __Modelo Relacional__

La unión de los dataframes `transactions` y `clients` es esencial para enriquecer el conjunto de datos transaccionales con información detallada de los clientes. Esto permite generar nuevas características valiosas para el análisis, como recencia, frecuencia y monto monetario (RFM), y entender mejor el comportamiento de compra. Al combinar ambas fuentes de datos, se puede realizar una segmentación más precisa de los clientes y mejorar la personalización de las recomendaciones. Además, la unión mediante un `left join` asegura que se mantengan todas las transacciones, incluso las de clientes nuevos, lo que facilita la identificación de brechas de información.

In [25]:
# Unir los dataframes
mdf = pd.merge(transactions, clients, on='ACCOUNT_ID', how='left')

In [ ]:
# Verificar que no hay valores nulos
mdf.isna().sum()

In [ ]:
# Numero de clientes nuevos sin información
mdf[mdf['CHANNEL'].isna()]['ACCOUNT_ID'].nunique()

Al revisar los valores nulos en el dataframe combinado (`mdf`), observamos que algunas columnas relacionadas con los clientes, como `BUSINESS_SEGMENT`, `TOTAL_VOLUME`, `DIFF_SKU_BY_ORDER`, y otras, tienen 6165 valores nulos. Esto indica que hay clientes nuevos en las transacciones que no tienen información previa en el dataframe de clientes. Específicamente, 231 clientes son nuevos y no cuentan con datos adicionales en el dataframe de clientes.

Dado que no tenemos datos suficientes sobre ellos, los excluiremos del modelo para evitar introducir ruido.

In [ ]:
# Eliminar valores nulos
clean_mdf = mdf.dropna(subset=['CHANNEL'])
clean_mdf.head()

## __Feature Engineering__

* #### __`Características Temporales`__

In [ ]:
# Crear columnas 'MES', 'DIA', 'DAY_OF_WEEK'
clean_mdf['MES'] = clean_mdf['INVOICE_DATE'].dt.month
clean_mdf['DIA'] = clean_mdf['INVOICE_DATE'].dt.day
clean_mdf['DAY_OF_WEEK'] = clean_mdf['INVOICE_DATE'].dt.dayofweek

In [ ]:
# Crear características de desfase
clean_mdf['LAG_1'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(1)
clean_mdf['LAG_2'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(2)
clean_mdf['LAG_3'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(3)
clean_mdf['LAG_4'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(4)
clean_mdf['LAG_5'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(5)
clean_mdf['LAG_6'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(6)
clean_mdf['LAG_7'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(7)
clean_mdf['LAG_8'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(8)
clean_mdf['LAG_9'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(9)
clean_mdf['LAG_10'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(10)
clean_mdf['LAG_11'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(11)
clean_mdf['LAG_12'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(12)
clean_mdf['LAG_13'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(13)
clean_mdf['LAG_14'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].shift(14)

# Crear medias moviles
clean_mdf['MA_3'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].transform(lambda x: x.rolling(3).mean())
clean_mdf['MA_6'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].transform(lambda x: x.rolling(6).mean())
clean_mdf['MA_9'] = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].transform(lambda x: x.rolling(9).mean())

In [ ]:
# Verificar que no hay valores nulos
clean_mdf.isna().sum()

In [ ]:
# Rellenar valores nulos
clean_mdf.fillna(0, inplace=True)

# Verificar que no hay valores nulos
clean_mdf.isna().sum()

In [ ]:
clean_mdf.shape

* #### __`Recencia, Frecuencia y Volumen`__

Dado el objetivo del proyecto, que es predecir qué productos comprará un cliente en su próximo pedido en la aplicación B2B BEES, el enfoque más adecuado es `calcular las métricas a nivel de cliente`. Debemos entender el comportamiento general de cada cliente para hacer recomendaciones personalizadas basadas en su historial de compras, en lugar de hacerlo a nivel de transacción.

Razones por que el enfoque a nivel de cliente es más adecuado:

* __`Recencia`__: A nivel de cliente, interesa saber cuánto tiempo ha pasado desde la última compra, ya que esto puede influir en la probabilidad de que realice otra compra pronto.

* __`Frecuencia`__: Saber cuántas órdenes distintas ha realizado cada cliente. Esto es clave para identificar patrones de compra, como si un cliente compra con regularidad o esporádicamente.

* __`Total de ítems comprados`__: A nivel de cliente, el volumen total de ítems comprados te permite identificar a clientes que hacen compras grandes o pequeñas, lo que es importante para determinar recomendaciones de productos.

In [34]:
from datetime import datetime

# Calcular la fecha máxima de transacciones
last_date = clean_mdf['INVOICE_DATE'].max()

# Calcular recencia
recency_df = clean_mdf.groupby('ACCOUNT_ID').agg(last_purchase=('INVOICE_DATE', 'max')).reset_index()
recency_df['RECENCIA'] = (last_date - recency_df['last_purchase']).dt.days

In [35]:
# Calcular la frecuencia
frequency_df = clean_mdf.groupby('ACCOUNT_ID')['ORDER_ID'].nunique().reset_index()
frequency_df.columns = ['ACCOUNT_ID', 'FRECUENCIA']

In [36]:
# Calcular el Volumen
monetary_df = clean_mdf.groupby('ACCOUNT_ID')['ITEMS_PHYS_CASES'].sum().reset_index()
monetary_df.columns = ['ACCOUNT_ID', 'TOTAL_ITEMS']

In [ ]:
rfm_df = pd.merge(recency_df[['ACCOUNT_ID', 'RECENCIA']],
                  frequency_df[['ACCOUNT_ID', 'FRECUENCIA']],
                  on='ACCOUNT_ID')
rfm_df = pd.merge(rfm_df, monetary_df[['ACCOUNT_ID', 'TOTAL_ITEMS']], on='ACCOUNT_ID')

rfm_df

In [ ]:
# Fusionar las tablas
df_merged = pd.merge(rfm_df, clean_mdf, how='inner', on='ACCOUNT_ID')

# Verificar el DataFrame resultante
df_merged.head()

In [ ]:
# Verificar las columnas
df_merged.columns

## __Análisis Exploratorio de Datos (EDA)__

## Variables Numéricas

In [ ]:
# Definir el tamaño de la figura para los gráficos
plt.figure(figsize=(16, 12))

# Crear el primer gráfico: Distribución de 'ITEMS_PHYS_CASES'
plt.subplot(2, 2, 1)
sns.histplot(df_merged['ITEMS_PHYS_CASES'], bins=50, kde=True, color='blue')
plt.title('Distribución de ITEMS_PHYS_CASES')
plt.xlabel('ITEMS_PHYS_CASES')
plt.ylabel('Frecuencia')

# Crear el segundo gráfico: Distribución de 'TOTAL_VOLUME'
plt.subplot(2, 2, 2)
sns.histplot(df_merged['TOTAL_VOLUME'], bins=50, kde=True, color='green')
plt.title('Distribución de TOTAL_VOLUME')
plt.xlabel('TOTAL_VOLUME')
plt.ylabel('Frecuencia')

# Crear el tercer gráfico: Distribución de 'DIFF_SKU_BY_ORDER'
plt.subplot(2, 2, 3)
sns.histplot(df_merged['DIFF_SKU_BY_ORDER'], bins=50, kde=True, color='red')
plt.title('Distribución de DIFF_SKU_BY_ORDER')
plt.xlabel('DIFF_SKU_BY_ORDER')
plt.ylabel('Frecuencia')

# Crear el cuarto gráfico: Distribución de 'TOTAL_DIFF_SKU'
plt.subplot(2, 2, 4)
sns.histplot(df_merged['TOTAL_DIFF_SKU'], bins=50, kde=True, color='purple')
plt.title('Distribución de TOTAL_DIFF_SKU')
plt.xlabel('TOTAL_DIFF_SKU')
plt.ylabel('Frecuencia')

# Ajustar el espacio entre los gráficos
plt.tight_layout()

# Mostrar los gráficos
plt.show()

In [ ]:
# Crear una figura con 4 subplots (2 filas, 2 columnas)
fig = make_subplots(rows=2, cols=2, subplot_titles=('Distribución de ITEMS_PHYS_CASES',
                                                    'Distribución de TOTAL_VOLUME',
                                                    'Distribución de DIFF_SKU_BY_ORDER',
                                                    'Distribución de TOTAL_DIFF_SKU'))

# Crear boxplot para ITEMS_PHYS_CASES
fig.add_trace(go.Box(y=df_merged['ITEMS_PHYS_CASES'], name='ITEMS_PHYS_CASES', marker_color='blue'), row=1, col=1)

# Crear boxplot para TOTAL_VOLUME
fig.add_trace(go.Box(y=df_merged['TOTAL_VOLUME'], name='TOTAL_VOLUME', marker_color='green'), row=1, col=2)

# Crear boxplot para DIFF_SKU_BY_ORDER
fig.add_trace(go.Box(y=df_merged['DIFF_SKU_BY_ORDER'], name='DIFF_SKU_BY_ORDER', marker_color='red'), row=2, col=1)

# Crear boxplot para TOTAL_DIFF_SKU
fig.add_trace(go.Box(y=df_merged['TOTAL_DIFF_SKU'], name='TOTAL_DIFF_SKU', marker_color='purple'), row=2, col=2)

# Actualizar el layout de la figura
fig.update_layout(height=600, width=800, title_text="Boxplots de Variables Seleccionadas", showlegend=False)

# Mostrar la figura
fig.show()

* __ITEMS_PHYS_CASES__: La mayoría de los valores se encuentran en rangos bajos, pero existen algunos valores atípicos significativos, que llegan hasta 2000. Esto indica que algunos clientes compran cantidades significativamente mayores de productos en comparación con el promedio.

* __TOTAL_VOLUME__: Al igual que en el caso anterior, la mayoría de los valores se concentran en un rango pequeño, con algunos outliers que superan los 4000. Esto también sugiere que hay casos de volúmenes de compra mucho más altos que los típicos.

* __DIFF_SKU_BY_ORDER__: La dispersión de esta variable es mucho menor. Aunque se observan algunos valores atípicos, la mayoría de los clientes compran entre 5 y 10 SKU diferentes por orden. Los valores atípicos no son tan extremos como en las variables anteriores.

* __TOTAL_DIFF_SKU__: La mayoría de los clientes tienen entre 27 y 64 SKU distintos, con algunos outliers que alcanzan los 150. Esto indica que ciertos clientes compran una variedad significativamente mayor de productos en comparación con el resto.

In [ ]:
# Top 10 productos más comprados
top_skus = df_merged['SKU_ID'].value_counts().head(10)
plt.figure(figsize=(10, 5))
top_skus.plot(kind='bar')
plt.title('Top 10 Productos (SKU) Más Comprados')
plt.xlabel('SKU ID')
plt.ylabel('Número de Compras')
plt.show()

Observamos que el producto con el SKU `7038` es el más popular, seguido de otros como el `19088`, `7651`, y `24880`, todos con un número significativo de compras. Esto indica una clara preferencia por ciertos productos, lo que puede ser clave para un sistema de recomendaciones, especialmente aquellos con comportamientos de compra similares.

## Variables Categóricas

In [ ]:
# Definir el tamaño de la figura para los gráficos
plt.figure(figsize=(16, 12))

# Gráfico 1: Compras por Concentración de negocios (CONCENTRATION)
plt.subplot(2, 2, 1)
compras_por_concentracion = df_merged['CONCENTRATION'].value_counts()
compras_por_concentracion.plot(kind='bar', color='blue')
plt.title('Distribución de Compras por Concentración')
plt.xlabel('Concentración')
plt.ylabel('Número de Compras')

# Gráfico 2: Compras por Nivel Socioeconómico (NSE)
plt.subplot(2, 2, 2)
compras_por_nse = df_merged['NSE'].value_counts()
compras_por_nse.plot(kind='bar', color='green')
plt.title('Distribución de Compras por NSE')
plt.xlabel('Nivel Socioeconómico')
plt.ylabel('Número de Compras')

# Gráfico 3: Compras por Segmento Único (UNIQUE_SEGMENT)
plt.subplot(2, 2, 3)
compras_por_segmento = df_merged['UNIQUE_SEGMENT'].value_counts()
compras_por_segmento.plot(kind='bar', color='red')
plt.title('Distribución de Compras por Segmento Único')
plt.xlabel('Segmento Único')
plt.ylabel('Número de Compras')

# Gráfico 4: Compras por Canal (CHANNEL)
plt.subplot(2, 2, 4)
compras_por_canal = df_merged['CHANNEL'].value_counts()
compras_por_canal.plot(kind='bar', color='purple')
plt.title('Distribución de Compras por Canal')
plt.xlabel('Canal')
plt.ylabel('Número de Compras')

# Ajustar el espacio entre los gráficos
plt.tight_layout()

# Mostrar los gráficos
plt.show()

La mayoría de los clientes pertenecen a los segmentos `HighUsage` y `MediumUsage` de productos, lo que refleja un comportamiento de consumo considerable. En cuanto a la concentración, los niveles altos y medios predominan entre los clientes.

El nivel socioeconómico `Medio` es el más común, con pocos clientes de nivel alto, lo que puede indicar una oportunidad para segmentar y personalizar ofertas para diferentes grupos.

El segmento `Activos` domina en términos de actividad, aunque hay un número considerable de `Inactivos` y otros segmentos que podrían ser objeto de campañas de reactivación o retención.

La mayoría de las transacciones provienen del canal `Tradicional`, lo que sugiere que este es el canal principal para la distribución de productos. Los `Kioscos/Maxikioscos` también representan una parte significativa del total, lo que indica que son un punto importante de venta.

### Distribución de la Recencia, Frecuencia y Total de Ítems Comprados

In [ ]:
# Ajustar número de bins para recencia
plt.figure(figsize=(10, 5))
sns.histplot(df_merged['RECENCIA'], bins=30, kde=True)
plt.title('Distribución de la Recencia (días desde la última compra)')
plt.xlabel('Recencia (días)')
plt.ylabel('Frecuencia')
plt.show()

# Ajustar número de bins para frecuencia
plt.figure(figsize=(10, 5))
sns.histplot(df_merged['FRECUENCIA'], bins=30, kde=True)
plt.title('Distribución de la Frecuencia (número de órdenes)')
plt.xlabel('Frecuencia (órdenes)')
plt.ylabel('Frecuencia')
plt.show()

# Ajustar número de bins para total_items
plt.figure(figsize=(10, 5))
sns.histplot(df_merged['TOTAL_ITEMS'], bins=50, kde=True)
plt.title('Distribución del Total de Ítems Comprados')
plt.xlabel('Total de Ítems Comprados')
plt.ylabel('Frecuencia')
plt.show()

* `Recencia`: La mayoría de los clientes son compradores recientes, lo cual es positivo para CMQ ya que sugiere que los clientes están activos en la plataforma BEES.

* `Frecuencia`: La mayoría de los clientes compra con una frecuencia moderada. Pocos clientes son altamente frecuentes (más de 40 órdenes), lo que podría ser un grupo interesante para análisis mas profundos.

* `Total de Ítems Comprados`: Hay una concentración de clientes que compran cantidades pequeñas, pero también algunos clientes grandes que compran cantidades significativas de productos, lo cual sugiere la presencia de clientes mayoristas.

In [ ]:
# Crear subplots: 1 fila, 3 columnas
fig = make_subplots(rows=1, cols=3, subplot_titles=("Recencia", "Frecuencia", "Total de Ítems Comprados"))

# Boxplot para Recencia
fig.add_trace(go.Box(y=df_merged['RECENCIA'], name="Recencia"), row=1, col=1)

# Boxplot para Frecuencia
fig.add_trace(go.Box(y=df_merged['FRECUENCIA'], name="Frecuencia"), row=1, col=2)

# Boxplot para Total de Ítems Comprados
fig.add_trace(go.Box(y=df_merged['TOTAL_ITEMS'], name="Total Ítems"), row=1, col=3)

# Actualizar el layout
fig.update_layout(title_text="Distribuciones de Recencia, Frecuencia y Total de Ítems Comprados", showlegend=False)

# Mostrar la figura
fig.show()

* `Clientes recientes`: La gran mayoría de los clientes son compradores recientes, la mediana indica que en general cada 2 días los clientes hacen compras.

* `Frecuencia`: Aunque la mediana de frecuencia es de 14 órdenes, hay algunos clientes que realizan muchas más compras (outliers).

* `Volumen total de ítems`: La mayoría de los clientes hacen pedidos pequeños con una media de 206, mientras que hay unos pocos clientes que hacen pedidos muy grandes.

### Patrones Temporales (MES, DIA, DAY_OF_WEEK)

In [ ]:
# Agrupar los datos por mes para contar las órdenes
orders_by_month = df_merged.groupby('MES')['ORDER_ID'].nunique().reset_index()

# Gráfico de tendencia del número de órdenes por mes
plt.figure(figsize=(10, 6))
sns.lineplot(x='MES', y='ORDER_ID', data=orders_by_month, marker='o')
plt.title('Número de Órdenes por Mes')
plt.xlabel('Mes')
plt.ylabel('Órdenes')
plt.xticks(ticks=orders_by_month['MES'], labels=['Mayo', 'Junio', 'Julio', 'Agosto'])
plt.grid(True)
plt.show()

# Compras por día de la semana
orders_by_day = df_merged.groupby('DAY_OF_WEEK')['ORDER_ID'].nunique().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(x='DAY_OF_WEEK', y='ORDER_ID', data=orders_by_day, marker='o')
plt.title('Órdenes por Día de la Semana')
plt.xlabel('Día de la Semana')
plt.ylabel('Órdenes')
plt.xticks(ticks=orders_by_day['DAY_OF_WEEK'], labels=['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado'])
plt.grid(True)
plt.show()

* __Crecimiento Sostenido__: 

- El crecimiento mensual en las órdenes indica una tendencia positiva, y sugiere que se pueden realizar más esfuerzos para continuar promoviendo el uso de la plataforma BEES en los próximos meses.

* __Patrones Semanales__:

- La mayor parte de las compras se concentran los miercoles y viernes, lo que puede indicar que las empresas planifican sus compras durante estos días para tener stock a lo largo de la semana.

- Los días jueves y sábado tienen una caída en la actividad, lo que sugiere que los esfuerzos logísticos y de marketing pueden ajustarse para concentrarse en los días de mayor actividad.

- Lunes tiene pocas órdenes, lo que puede indicar que las empresas suelen empezar a planificar sus compras más adelante en la semana.

In [ ]:
# Agrupar los datos por día para contar las órdenes
orders_by_day_2 = df_merged.groupby('INVOICE_DATE')['ORDER_ID'].nunique().reset_index()

# Gráfico de tendencia del número de órdenes por día
fig = px.line(orders_by_day_2, x='INVOICE_DATE', y='ORDER_ID', title='Número de Órdenes por Día',
              labels={'INVOICE_DATE': 'Fecha', 'ORDER_ID': 'Órdenes'}, markers=True)

# Ajustar el formato de la fecha en el eje X para mejor visualización
fig.update_xaxes(tickformat='%Y-%m-%d', tickangle=45)
fig.show()

* `Patrón Semanal`: Se observa un patrón cíclico que se repite aproximadamente cada semana, con un aumento en las órdenes seguido de una caída generalmente los Lunes. Esto podría deberse a la naturaleza del negocio, donde ciertos días de la semana tienen más actividad (los miercoles durante la semana y viernes al terminar la semana).

* `Picos de Actividad`: Hay picos pronunciados de órdenes, lo que sugiere que hay días en los que los clientes tienden a realizar más compras, lo que podría estar relacionado con promociones, hábitos de compra o fechas específicas de alta demanda.

* `Tendencias Estacionales`: No parece haber una tendencia estacional fuerte (incremento o decremento constante), sino que los picos y caídas se mantienen relativamente consistentes a lo largo del tiempo.

In [ ]:
# Agrupar los datos por día para contar las órdenes
orders_by_day = df_merged.groupby('INVOICE_DATE')['ORDER_ID'].nunique().reset_index()

# Agrupar las órdenes por fecha
orders_by_day = df_merged.groupby('INVOICE_DATE').size().reset_index(name='orders_per_day')

# Crear características de desfase para el número de órdenes del día anterior (lag 1)
orders_by_day['lag_1'] = orders_by_day['orders_per_day'].shift(1)

# Crear características de desfase para días anteriores hasta completar 2 semanas
orders_by_day['lag_2'] = orders_by_day['orders_per_day'].shift(2)
orders_by_day['lag_3'] = orders_by_day['orders_per_day'].shift(3)
orders_by_day['lag_4'] = orders_by_day['orders_per_day'].shift(4)
orders_by_day['lag_5'] = orders_by_day['orders_per_day'].shift(5)
orders_by_day['lag_6'] = orders_by_day['orders_per_day'].shift(6)
orders_by_day['lag_7'] = orders_by_day['orders_per_day'].shift(7)
orders_by_day['lag_8'] = orders_by_day['orders_per_day'].shift(8)
orders_by_day['lag_9'] = orders_by_day['orders_per_day'].shift(9)
orders_by_day['lag_10'] = orders_by_day['orders_per_day'].shift(10)
orders_by_day['lag_11'] = orders_by_day['orders_per_day'].shift(11)
orders_by_day['lag_12'] = orders_by_day['orders_per_day'].shift(12)
orders_by_day['lag_13'] = orders_by_day['orders_per_day'].shift(13)
orders_by_day['lag_14'] = orders_by_day['orders_per_day'].shift(14)

# Crear características de media móvil para 2,3,4,5,6,7 días
orders_by_day['ma_3'] = orders_by_day['orders_per_day'].rolling(window=3).mean()
orders_by_day['ma_6'] = orders_by_day['orders_per_day'].rolling(window=6).mean()
orders_by_day['ma_9'] = orders_by_day['orders_per_day'].rolling(window=9).mean()
orders_by_day['ma_14'] = orders_by_day['orders_per_day'].rolling(window=14).mean()

# Mostrar el resultado
orders_by_day

In [ ]:
# Crear la figura
fig = go.Figure()

# Asegúrate de que el DataFrame orders_by_day tiene las columnas 'INVOICE_DATE', 'orders_per_day', y 'ma_6'

# Agregar la serie de tiempo original (órdenes por día)
fig.add_trace(go.Scatter(x=orders_by_day['INVOICE_DATE'], y=orders_by_day['orders_per_day'],
                         mode='lines', name='Órdenes por Día'))

# Agregar la media móvil de 6 días
fig.add_trace(go.Scatter(x=orders_by_day['INVOICE_DATE'], y=orders_by_day['ma_6'],
                         mode='lines', name='Media Móvil 6 días', line=dict(color='green')))

# Títulos y ajustes
fig.update_layout(title='Tendencia de Órdenes por Día con Medias Móviles',
                  xaxis_title='Fecha',
                  yaxis_title='Número de Órdenes',
                  template='plotly_white')

# Mostrar la gráfica
fig.show()

Las fluctuaciones diarias son significativas, pero la media móvil muestra que, en general, el comportamiento de las órdenes es estable y con tendencia al alza. 

In [50]:
# Establecer INVOICE_DATE como índice
orders_by_day.set_index('INVOICE_DATE', inplace=True)

# Eliminar valores nulos
orders_by_day.dropna(inplace=True)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Descomponer la serie temporal remuestreada (usar la columna específica)
decomposed = seasonal_decompose(orders_by_day['orders_per_day'], model='additive', period=7)

# Filtrar el componente estacional para el rango de fechas deseado (21 de julio de 2022 al 6 de agosto de 2022)
seasonal_filtered = decomposed.seasonal['2022-07-04':'2022-07-21']

# Crear el gráfico en Plotly
fig = go.Figure()

# Añadir la línea del componente estacional
fig.add_trace(go.Scatter(x=seasonal_filtered.index, y=seasonal_filtered, mode='lines', name='Componente Estacional'))

# Títulos y etiquetas
fig.update_layout(
    title='Componente Estacional - 4 al 21 de Julio de 2022',
    xaxis_title='Fecha',
    yaxis_title='Número de Órdenes',
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()

El gráfico del componente estacional revela un patrón cíclico claro en las órdenes del 4 al 21 de Julio de 2022, con picos y valles recurrentes. Estos ciclos sugieren que ciertos días de la semana tienen mayor demanda, posiblemente influenciados por eventos específicos o comportamientos de compra regulares.

## __Data Wrangling y Modelado__

#### __Preprocesamiento de Datos__

El escalado de las variables numéricas como `TOTAL_VOLUME`, `DIFF_SKU_BY_ORDER`, `TOTAL_DIFF_SKU`, `RECENCIA`, `FRECUENCIA` y `MONETARY` es crucial para garantizar que los modelos de aprendizaje automático, como `SVD` y `KNN`, funcionen correctamente. Estas variables pueden tener diferentes rangos de valores y magnitudes, lo que podría afectar negativamente el rendimiento del modelo, especialmente en algoritmos basados en distancias como KNN. Al aplicar un escalado estándar, transformamos estas variables para que tengan una media de 0 y una desviación estándar de 1, lo que asegura que todas las características numéricas tengan la misma importancia y escala en el proceso de entrenamiento, evitando que las variables con valores grandes dominen el modelo. Esto también mejora la estabilidad y precisión de las predicciones en los modelos supervisados y de filtrado colaborativo.

In [ ]:
# Verificar si las columnas existen en df_merged antes de aplicar el escalado
numeric_cols = ['TOTAL_VOLUME', 'DIFF_SKU_BY_ORDER', 'TOTAL_DIFF_SKU', 'RECENCIA', 'FRECUENCIA', 'MONETARY'] 
numeric_cols_present = [col for col in numeric_cols if col in df_merged.columns]

# Si las columnas están presentes, proceder con el escalado
if numeric_cols_present:
    standard_scaler = StandardScaler()
    df_merged[numeric_cols_present] = standard_scaler.fit_transform(df_merged[numeric_cols_present])
else:
    print(f"Las columnas {numeric_cols} no están presentes en el DataFrame.")

# Mostrar las primeras filas para verificar los resultados
df_merged.head()

Vamos a normalizar las variables categoricas utilizando `OneHotEncoder` ya que crea una representación binaria adecuada para capturar la naturaleza de las variables categóricas sin asumir un orden implícito.

In [ ]:
# Realizar One-Hot Encoding en las columnas categóricas
df_merged = pd.get_dummies(df_merged, columns=['BUSINESS_SEGMENT', 'CONCENTRATION', 'NSE', 'UNIQUE_SEGMENT', 'CHANNEL'], drop_first=True)

df_merged.head()

Eliminamos las columnas que no son relevantes para los modelos basados en sistemas de recomendación.

In [ ]:
# Eliminar las columnas 'ORDER_ID' e 'INVOICE_DATE'
df_merged = df_merged.drop(columns=['ORDER_ID', 'INVOICE_DATE'])

# Verificar que se han eliminado correctamente
print(df_merged.columns) 

Haremos una division del DataFrame en los meses de ``junio``, ``julio`` y ``agosto``, por la necesidad de evaluar el comportamiento de las predicciones de los modelos en periodos de tiempo específicos. Al segmentar los datos, podemos obtener información detallada sobre patrones de compra y variaciones en cada mes, lo cual es crucial en un sistema de recomendación. Estas son las razones principales para dividir el DataFrame por meses:

``Evaluación del modelo en distintos periodos``: Al dividir los datos, podemos analizar si los modelos, como ``KNN`` o ``SVD``, funcionan de manera consistente a lo largo del tiempo o si existen diferencias significativas en su rendimiento en distintos meses.

``Identificación de patrones estacionales o cambios de comportamiento``: Algunos productos pueden tener mayor demanda en un mes específico debido a factores estacionales, promociones o eventos. Dividir los datos por meses nos permite captar estos patrones que podrían mejorar las recomendaciones.

``Validación temporal del modelo``: Al trabajar con datos históricos de compras, es útil dividir los datos cronológicamente para entrenar el modelo con un periodo (por ejemplo, junio-julio) y validarlo o testearlo con otro (agosto). Esto simula el comportamiento real del sistema en producción, donde las recomendaciones se generan basadas en el historial previo.

``Predicciones más precisas``: Dividir los datos en diferentes meses permite ajustar mejor las predicciones a las condiciones del mercado o del cliente en un tiempo específico, lo que puede mejorar la personalización de las recomendaciones para cada cliente en función de los productos que fueron relevantes en ese mes.

In [55]:
# Dividir el DataFrame en conjuntos de datos para cada mes
june_df = df_merged[df_merged['MES'] == 6] 
july_df = df_merged[df_merged['MES'] == 7]
august_df = df_merged[df_merged['MES'] == 8]

In [ ]:
# Verificar las dimensiones de los conjuntos de datos
print(f"Datos de junio: {june_df.shape}")
print(f"Datos de julio: {july_df.shape}")
print(f"Datos de agosto: {august_df.shape}")

Vamos a identificar clientes que realizaron compras en `julio` y `agosto`, lo que permite analizar su comportamiento recurrente y mejorar las predicciones de compra.

Al centrarse en estos clientes, se pueden comparar sus compras entre ambos meses para detectar cambios en preferencias o la efectividad de promociones.

Este enfoque mejora las recomendaciones personalizadas, basándose en patrones consistentes de compra.

Además, facilita la evaluación del modelo, ya que permite entrenarlo con datos de julio y validarlo con las compras de agosto.

In [ ]:
# Obtener los clientes que compraron tanto en julio como agosto
clientes_julio = july_df['ACCOUNT_ID'].unique()
clientes_agosto = august_df['ACCOUNT_ID'].unique()

# Obtener la intersección de ambos conjuntos de clientes
clientes_ambos_meses = list(set(clientes_julio).intersection(set(clientes_agosto)))

# Filtrar los DataFrames para considerar solo esos clientes
july_df_filtrado = july_df[july_df['ACCOUNT_ID'].isin(clientes_ambos_meses)]
august_df_filtrado = august_df[august_df['ACCOUNT_ID'].isin(clientes_ambos_meses)]

print(f"Número de clientes en ambos meses: {len(clientes_ambos_meses)}")

## __Desarrollo de Modelos para Sistemas de Recomendación__

Para decidir si aplicar ``SVD`` (Singular Value Decomposition) o ``KNN`` (K-Nearest Neighbors) primero, es útil entender cómo cada enfoque puede resolver el problema del sistema de recomendación. Ambos enfoques son populares, pero tienen diferentes fortalezas:

1. __``SVD (Singular Value Decomposition)``__:


__Ventajas__:


* __Recomendación colaborativa__: Es un enfoque basado en la descomposición de matrices, ideal para sistemas de recomendación colaborativa. Utiliza patrones en las compras de clientes y productos para hacer predicciones sobre qué productos comprarán los clientes en el futuro.

* __Escalabilidad__: SVD es más eficiente y puede manejar grandes datasets, lo que lo hace ideal si tienes muchos productos (SKU) y clientes.

* __Predicción precisa__: Es muy bueno para capturar relaciones latentes entre productos y clientes.

* __Aplicabilidad__: Recomendable si tienes datos dispersos, es decir, muchos clientes compran solo algunos productos, pero no todos. En este caso, SVD puede aprovechar bien esos patrones.


2. __``KNN (K-Nearest Neighbors)``__:


__Ventajas__:


* __Simplicidad__: KNN es fácil de implementar y puede ser eficaz para problemas con datos no tan grandes o donde la relación entre clientes y productos es más directa.

* __Recomendación basada en similitud__: KNN puede recomendar productos basados en la similitud entre clientes o productos, comparando comportamientos de compra similares.

* __Recomendación basada en proximidad__: Recomendaciones directas basadas en productos similares a los que ya compró el cliente.

* __Aplicabilidad__: Recomendable si tienes un dataset moderado o si buscas identificar productos similares basados en patrones de compra de los clientes más cercanos (vecinos).


In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split

# Cargar datos para el modelo
reader = Reader(rating_scale=(1, 5))  # Ajustar según la escala de tu conjunto de datos
data = Dataset.load_from_df(july_df_filtrado[['ACCOUNT_ID', 'SKU_ID', 'ITEMS_PHYS_CASES']], reader)

# Dividir en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2)

# Entrenar modelo SVD
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Evaluar el rendimiento en el conjunto de prueba
predictions = modelo_svd.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

In [ ]:
from surprise import KNNBasic

# Opciones de similitud para KNN user-based
sim_options_user = {
    'name': 'cosine',        # Puede ser 'cosine' o 'pearson'
    'user_based': True       # True para user-based
}

# Opciones de similitud para KNN item-based
sim_options_item = {
    'name': 'cosine',        # Puede ser 'cosine' o 'pearson'
    'user_based': False      # False para item-based
}

# Modelo KNN para user-based
modelo_knn_user = KNNBasic(sim_options=sim_options_user)
modelo_knn_user.fit(trainset)

# Hacer predicciones y calcular RMSE para KNN user-based
predicciones_knn_user = modelo_knn_user.test(testset)
rmse_knn_user = accuracy.rmse(predicciones_knn_user)
print(f"RMSE del modelo KNN (user-based): {rmse_knn_user}")

# Modelo KNN para item-based
modelo_knn_item = KNNBasic(sim_options=sim_options_item)
modelo_knn_item.fit(trainset)

# Hacer predicciones y calcular RMSE para KNN item-based
predicciones_knn_item = modelo_knn_item.test(testset)
rmse_knn_item = accuracy.rmse(predicciones_knn_item)
print(f"RMSE del modelo KNN (item-based): {rmse_knn_item}")

## __Modelo K-Nearest Neighbors (KNN)__

El modelo ``K-Nearest Neighbors (KNN)`` es utilizado para generar recomendaciones basadas en la similitud entre productos (item-based) y entre usuarios (user-based). El flujo de trabajo incluye los siguientes pasos:

1. __Preparación de los datos__: Los datos de julio se convierten al formato necesario para el uso en el modelo KNN utilizando la biblioteca surprise. Esto incluye definir las columnas clave como ``ACCOUNT_ID``, ``SKU_ID`` y ``ITEMS_PHYS_CASES``.

2. __Entrenamiento de los modelos KNN__: Se entrenan dos modelos, uno basado en la similitud de productos y otro basado en la similitud de usuarios. La métrica utilizada es el coseno (``cosine similarity``), que es adecuada para medir la similitud en datos dispersos como estos.

3. __Generación de predicciones__: Una vez entrenados los modelos, se generan predicciones con el modelo ``KNN (item-based)`` para evaluar su rendimiento en un conjunto de datos de prueba de agosto.

4. __Ajuste de predicciones__: Las predicciones se ajustan considerando características adicionales del cliente, como recencia, frecuencia, y segmentación de negocio. Estas características se normalizan y se combinan para obtener un puntaje ajustado que mejora la recomendación final.

5. __Evaluación de las recomendaciones__: Finalmente, se mide la precisión y el recall del modelo. La precisión representa la proporción de productos recomendados que fueron comprados, mientras que el recall mide la proporción de productos comprados que fueron recomendados.

In [60]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic, accuracy
from surprise.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Convertir july_df_filtrado a formato largo para Surprise
def preparar_datos(df, rating_col='ITEMS_PHYS_CASES'):
    reader = Reader(rating_scale=(1, df[rating_col].max()))
    data = Dataset.load_from_df(df[['ACCOUNT_ID', 'SKU_ID', rating_col]], reader)
    return data

# Preparamos los datos de julio
data_july = preparar_datos(july_df_filtrado)

# Dividir los datos en conjuntos de entrenamiento y prueba
trainset_july, testset_july = train_test_split(data_july, test_size=0.2)

In [ ]:
# Definir las opciones de similitud para KNN (item-based y user-based)
sim_options_item = {'name': 'cosine', 'user_based': False}
sim_options_user = {'name': 'cosine', 'user_based': True}

# Crear los modelos KNN
modelo_knn_item = KNNBasic(sim_options=sim_options_item)
modelo_knn_user = KNNBasic(sim_options=sim_options_user)

# Entrenar los modelos con los datos de julio
modelo_knn_item.fit(trainset_july)
modelo_knn_user.fit(trainset_july)

# Generar predicciones para KNN (Item-based)
predicciones_knn_item_august = modelo_knn_item.test(testset_july)

### Recomendaciones:

Para el cliente especificado (`cliente_id`), el código genera recomendaciones excluyendo los productos que ya compró en julio y selecciona los top N productos con las mejores predicciones.

__Salida en Formato DataFrame__:

El resultado se presenta como un DataFrame con las columnas `ACCOUNT_ID`, `SKU_ID`, `SCORE`, y `FECHA_DE_RECOMENDACION`, mostrando las recomendaciones para ese cliente en particular.

__Visualización del DataFrame__:

El código imprime el DataFrame que contiene las recomendaciones para el cliente.

__Ejemplo de Salida Esperada__:

| ACCOUNT_ID | SKU_ID | SCORE | FECHA_DE_RECOMENDACION |
|------------|--------|-------|------------------------|
| 12345      | 7890   | 0.85  | 2024-09-18             |
| 12345      | 2345   | 0.78  | 2024-09-18             |
| 12345      | 6789   | 0.65  | 2024-09-18             |
| ...        | ...    | ...   | 2024-09-18             |

In [62]:
def ajustar_predicciones_knn(predicciones, df_caracteristicas, account_id, top_n=10):
    pred_df = pd.DataFrame(predicciones, columns=['ACCOUNT_ID', 'SKU_ID', 'rating_real', 'rating_pred', 'details'])

    # Filtrar solo las predicciones para el cliente especificado
    pred_df = pred_df[pred_df['ACCOUNT_ID'] == account_id]
    pred_df = pred_df.merge(df_caracteristicas, on=['ACCOUNT_ID', 'SKU_ID'], how='left')

    # Normalizar algunas variables clave para su uso en el ajuste
    pred_df['RECENCIA_NORMALIZADA'] = 1 / (1 + pred_df['RECENCIA'].clip(lower=0))
    pred_df['FRECUENCIA_NORMALIZADA'] = pred_df['FRECUENCIA'] / pred_df['FRECUENCIA'].max()
    pred_df['TOTAL_ITEMS_NORMALIZADA'] = pred_df['TOTAL_ITEMS'] / pred_df['TOTAL_ITEMS'].max()

    # Ajustar las predicciones utilizando una combinación de variables importantes
    pred_df['SCORE_AJUSTADO'] = pred_df['rating_pred'] * (
        pred_df['RECENCIA_NORMALIZADA'] * 0.2 + 
        pred_df['FRECUENCIA_NORMALIZADA'] * 0.3 + 
        pred_df['TOTAL_ITEMS_NORMALIZADA'] * 0.2 +
        pred_df['MA_3'] * 0.1 + 
        pred_df['LAG_1'] * 0.1 +
        pred_df['BUSINESS_SEGMENT_MediumUsage'] * 0.05 + 
        pred_df['CHANNEL_Mayorista'] * 0.05
    )

    # Clip para asegurar que los puntajes ajustados no sean negativos
    pred_df['SCORE_AJUSTADO'] = pred_df['SCORE_AJUSTADO'].clip(lower=0)

    # Eliminar duplicados de SKU_ID para el mismo ACCOUNT_ID
    pred_df = pred_df.drop_duplicates(subset=['ACCOUNT_ID', 'SKU_ID'])

    # Ordenar por SCORE_AJUSTADO en orden descendente y seleccionar el top_n
    pred_df = pred_df.sort_values(by='SCORE_AJUSTADO', ascending=False).head(top_n)

    # Añadir columna de fecha de recomendación
    pred_df['FECHA_DE_RECOMENDACION'] = datetime.now().strftime('%Y-%m-%d')

    return pred_df[['ACCOUNT_ID', 'SKU_ID', 'SCORE_AJUSTADO', 'FECHA_DE_RECOMENDACION']]

In [ ]:
# Ejemplo de uso:
cliente_id = 456111   # Input del cliente específico

# Ajustar las predicciones de KNN
predicciones_ajustadas_knn = ajustar_predicciones_knn(predicciones_knn_item_august, august_df_filtrado, cliente_id, top_n=10)

# Mostrar las recomendaciones ajustadas con KNN
print(predicciones_ajustadas_knn)

In [ ]:
def evaluar_recomendaciones(predicciones_ajustadas, df_real, cliente_id):
    productos_comprados = df_real[df_real['ACCOUNT_ID'] == cliente_id]['SKU_ID'].unique()
    productos_recomendados = predicciones_ajustadas['SKU_ID'].unique()
    aciertos = [sku for sku in productos_recomendados if sku in productos_comprados]

    precision = len(aciertos) / len(productos_recomendados) if productos_recomendados.size > 0 else 0
    recall = len(aciertos) / len(productos_comprados) if productos_comprados.size > 0 else 0
    return precision, recall

# Evaluar recomendaciones ajustadas
precision_knn, recall_knn = evaluar_recomendaciones(predicciones_ajustadas_knn, august_df_filtrado, cliente_id)
print(f"Precisión del modelo KNN: {precision_knn:.2f}, Recall del modelo KNN: {recall_knn:.2f}")

__Evaluación del modelo KNN__: En este caso, la precisión del modelo ``KNN`` fue del ``100%``, lo que indica que todos los productos recomendados fueron comprados. Sin embargo, el recall fue del ``40%``, lo que sugiere que el modelo no capturó todas las posibles compras realizadas por el cliente, lo que podría indicar que el modelo está recomendando un conjunto limitado de productos.

Estos resultados pueden mejorarse ajustando los hiperparámetros del modelo KNN o integrando más información sobre el comportamiento del cliente para capturar mejor la diversidad de productos comprados.

## Singular Value Decomposition (SVD)

1. __Función para generar recomendaciones con ``SVD``__:

* Se filtran los productos que el cliente no compró en el mes anterior.

* El modelo ``SVD`` predice los productos más relevantes basándose en las interacciones previas.

* Las recomendaciones se almacenan en un DataFrame con las columnas ``ACCOUNT_ID``, ``SKU_ID``, ``SCORE``, y ``FECHA_DE_RECOMENDACION``.

2. __Evaluación de las recomendaciones con ``SVD``__:

* Se evalúa el rendimiento de las recomendaciones midiendo la precisión y el recall.

* La precisión refleja la proporción de productos recomendados que fueron comprados por el cliente.

* El recall mide la proporción de productos comprados que fueron recomendados.

3. __Generación de recomendaciones combinadas con ``KNN (Item-based y User-based)``__:

* Se generan dos conjuntos de predicciones: uno utilizando la similitud entre items (productos similares) y otro utilizando la similitud entre usuarios (clientes con patrones de compra similares).

* Las predicciones se ajustan utilizando características adicionales del cliente, como ``recencia``, ``frecuencia`` y ``volumen total`` de compras, lo que ayuda a refinar las recomendaciones.

4. __Evaluación de las recomendaciones con ``KNN``__:

* Al igual que en el caso de ``SVD``, se calculan las métricas de ``precisión`` y ``recall`` para evaluar el rendimiento del modelo ``KNN`` combinado.

In [65]:
# Función para generar recomendaciones usando SVD
def generar_recomendaciones_para_cliente(modelo, cliente_id, productos, df_julio, top_n=10):
    productos_comprados_julio = df_julio[df_julio['ACCOUNT_ID'] == cliente_id]['SKU_ID'].unique()
    productos_no_comprados = [producto for producto in productos if producto not in productos_comprados_julio]

    predicciones = [modelo.predict(cliente_id, producto_id) for producto_id in productos_no_comprados]
    predicciones.sort(key=lambda x: x.est, reverse=True)
    recomendaciones = predicciones[:top_n]
    
    fecha_recomendacion = datetime.now().strftime('%Y-%m-%d')
    output = pd.DataFrame({
        'ACCOUNT_ID': [cliente_id] * top_n,
        'SKU_ID': [rec.iid for rec in recomendaciones],
        'SCORE': [rec.est for rec in recomendaciones],
        'FECHA_DE_RECOMENDACION': [fecha_recomendacion] * top_n
    })
    return output

In [ ]:
# Ejemplo de uso
cliente_id = 456111
productos = august_df_filtrado['SKU_ID'].unique()
df_recomendaciones_cliente_svd = generar_recomendaciones_para_cliente(modelo_svd, cliente_id, productos, july_df_filtrado, top_n=10)

# Mostrar las recomendaciones iniciales generadas por SVD
print("Recomendaciones iniciales:")
print(df_recomendaciones_cliente_svd)

In [ ]:
# Evaluar las recomendaciones
def evaluar_recomendaciones(predicciones_ajustadas, df_agosto, cliente_id):
    # Obtener los productos comprados en agosto por el cliente
    productos_comprados_agosto = df_agosto[df_agosto['ACCOUNT_ID'] == cliente_id]['SKU_ID'].unique()
    
    # Verificar si la columna SKU_ID existe y contiene datos
    if 'SKU_ID' not in predicciones_ajustadas.columns or predicciones_ajustadas['SKU_ID'].isnull().all():
        print("Error: No se encuentran recomendaciones válidas en las predicciones ajustadas.")
        return 0, 0

    # Obtener los productos recomendados
    productos_recomendados = predicciones_ajustadas['SKU_ID'].dropna().unique()
    
    # Verificar si hay productos recomendados y comprados
    if len(productos_recomendados) == 0 or len(productos_comprados_agosto) == 0:
        print("No hay productos recomendados o comprados para evaluar.")
        return 0, 0
    
    # Obtener los aciertos entre los productos recomendados y los comprados
    aciertos = [sku for sku in productos_recomendados if sku in productos_comprados_agosto]

    # Calcular precisión (productos recomendados comprados / productos recomendados)
    precision = len(aciertos) / len(productos_recomendados) if len(productos_recomendados) > 0 else 0

    # Calcular recall (productos recomendados comprados / productos comprados en agosto)
    recall = len(aciertos) / len(productos_comprados_agosto) if len(productos_comprados_agosto) > 0 else 0

    return precision, recall

# Evaluar las recomendaciones para el cliente
precision, recall = evaluar_recomendaciones(predicciones_ajustadas_knn, august_df_filtrado, cliente_id)
print(f"Precisión: {precision:.2f}, Recall: {recall:.2f}")

In [68]:
from datetime import datetime
import pandas as pd

def generar_recomendaciones_combinadas_ajustadas(modelo_item, modelo_user, cliente_id, productos, df_julio, df_caracteristicas, top_n=10):
    """
    Genera recomendaciones combinadas utilizando KNN (item-based y user-based),
    ajustando con características adicionales como RFM, medios móviles, y comportamiento del producto.
    """
    # Filtrar los productos comprados por el cliente en julio
    productos_comprados_julio = df_julio[df_julio['ACCOUNT_ID'] == cliente_id]['SKU_ID'].unique()

    # Generar recomendaciones con KNN (item-based)
    predicciones_item = [modelo_item.predict(cliente_id, producto_id) for producto_id in productos_comprados_julio]

    # Generar recomendaciones con KNN (user-based)
    predicciones_user = [modelo_user.predict(cliente_id, producto_id) for producto_id in productos_comprados_julio]

    # Combinar ambas listas de predicciones sin duplicar SKU
    recomendaciones_combinadas = {rec.iid: rec.est for rec in predicciones_item}
    for rec in predicciones_user:
        if rec.iid not in recomendaciones_combinadas:
            recomendaciones_combinadas[rec.iid] = rec.est

    # Crear DataFrame con las recomendaciones
    df_recomendaciones = pd.DataFrame(recomendaciones_combinadas.items(), columns=['SKU_ID', 'SCORE'])

    # Incorporar las características del cliente desde df_caracteristicas
    df_recomendaciones = df_recomendaciones.merge(df_caracteristicas[df_caracteristicas['ACCOUNT_ID'] == cliente_id], on='SKU_ID', how='left')

    # Verificar si hay valores faltantes en las columnas de ajuste
    columnas_ajuste = ['RECENCIA', 'FRECUENCIA', 'TOTAL_ITEMS', 'MA_3', 'LAG_1', 'BUSINESS_SEGMENT_MediumUsage', 'CHANNEL_Mayorista']
    if df_recomendaciones[columnas_ajuste].isnull().any().any():
        print("Existen valores nulos en las columnas de ajuste. Rellenando con valores predeterminados...")
        df_recomendaciones['RECENCIA'] = df_recomendaciones['RECENCIA'].fillna(0)
        df_recomendaciones['FRECUENCIA'] = df_recomendaciones['FRECUENCIA'].fillna(1)
        df_recomendaciones['TOTAL_ITEMS'] = df_recomendaciones['TOTAL_ITEMS'].fillna(1)
        df_recomendaciones['MA_3'] = df_recomendaciones['MA_3'].fillna(0)
        df_recomendaciones['LAG_1'] = df_recomendaciones['LAG_1'].fillna(0)
        df_recomendaciones['BUSINESS_SEGMENT_MediumUsage'] = df_recomendaciones['BUSINESS_SEGMENT_MediumUsage'].fillna(0)
        df_recomendaciones['CHANNEL_Mayorista'] = df_recomendaciones['CHANNEL_Mayorista'].fillna(0)

    # Normalizar las variables clave para el ajuste
    df_recomendaciones['RECENCIA_NORMALIZADA'] = 1 / (1 + df_recomendaciones['RECENCIA'].clip(lower=0))
    df_recomendaciones['FRECUENCIA_NORMALIZADA'] = df_recomendaciones['FRECUENCIA'] / df_recomendaciones['FRECUENCIA'].max()
    df_recomendaciones['TOTAL_ITEMS_NORMALIZADO'] = df_recomendaciones['TOTAL_ITEMS'] / df_recomendaciones['TOTAL_ITEMS'].max()

    # Ajustar el SCORE utilizando una combinación de las variables importantes
    df_recomendaciones['SCORE_AJUSTADO'] = df_recomendaciones['SCORE'] * (
        df_recomendaciones['RECENCIA_NORMALIZADA'] * 0.2 + 
        df_recomendaciones['FRECUENCIA_NORMALIZADA'] * 0.3 + 
        df_recomendaciones['TOTAL_ITEMS_NORMALIZADO'] * 0.2 +
        df_recomendaciones['MA_3'] * 0.1 + 
        df_recomendaciones['LAG_1'] * 0.1 +
        df_recomendaciones['BUSINESS_SEGMENT_MediumUsage'] * 0.05 + 
        df_recomendaciones['CHANNEL_Mayorista'] * 0.05
    )

    # Clip para asegurar que los puntajes ajustados no sean negativos
    df_recomendaciones['SCORE_AJUSTADO'] = df_recomendaciones['SCORE_AJUSTADO'].clip(lower=0)

    # Eliminar duplicados de SKU_ID
    df_recomendaciones = df_recomendaciones.drop_duplicates(subset=['SKU_ID'])

    # Completar ACCOUNT_ID faltante
    df_recomendaciones['ACCOUNT_ID'] = cliente_id

    # Ordenar por SCORE_AJUSTADO en orden descendente y seleccionar el top_n
    df_recomendaciones = df_recomendaciones.sort_values(by='SCORE_AJUSTADO', ascending=False).head(top_n)

    # Resetear el índice para el DataFrame final
    df_recomendaciones = df_recomendaciones.reset_index(drop=True)

    return df_recomendaciones[['ACCOUNT_ID', 'SKU_ID', 'SCORE_AJUSTADO']]

In [ ]:
# Ejemplo de uso:
cliente_id = 456111  # ID del cliente específico
productos = july_df['SKU_ID'].unique()  # Lista de productos del mes de junio para la base de recomendaciones

# Generar el top N de recomendaciones combinadas y ajustadas
df_recomendaciones_cliente = generar_recomendaciones_combinadas_ajustadas(
    modelo_knn_item, modelo_knn_user, cliente_id, productos, july_df_filtrado, august_df_filtrado, top_n=10)

# Mostrar las recomendaciones para el cliente
print(df_recomendaciones_cliente)

In [70]:
def evaluar_recomendaciones(df_recomendaciones, df_agosto, cliente_id):
    """
    Evalúa las recomendaciones revisando si el cliente compró los productos en agosto.
    Calcula precisión y recall.
    """
    # Obtener los productos comprados por el cliente en agosto
    productos_comprados_agosto = df_agosto[df_agosto['ACCOUNT_ID'] == cliente_id]['SKU_ID'].unique()

    # Obtener los productos recomendados
    productos_recomendados = df_recomendaciones['SKU_ID'].unique()

    # Verificar si las recomendaciones están entre los productos comprados
    aciertos = [sku for sku in productos_recomendados if sku in productos_comprados_agosto]

    # Calcular precisión (proporción de productos recomendados que fueron comprados)
    precision = len(aciertos) / len(productos_recomendados) if len(productos_recomendados) > 0 else 0

    # Calcular recall (proporción de productos comprados que fueron recomendados)
    recall = len(aciertos) / len(productos_comprados_agosto) if len(productos_comprados_agosto) > 0 else 0

    return precision, recall

In [ ]:
precision, recall = evaluar_recomendaciones(df_recomendaciones_cliente, august_df_filtrado, cliente_id)
print(f"Precisión: {precision:.2f}, Recall: {recall:.2f}")

Este enfoque híbrido logra una mejor personalización al combinar dos métodos complementarios. ``KNN es útil para encontrar productos o usuarios similares``, mientras que ``SVD captura patrones latentes en los datos``. La normalización y el ajuste basado en características del cliente ayudan a optimizar las recomendaciones, obteniendo una ``precisión`` de ``0.90`` y un ``recall`` de ``0.6``, lo que indica que el modelo logra recomendar con acierto una gran parte de los productos que el cliente finalmente compra.

## __BONUS TRACK__: Clustering de características RFM

In [ ]:
# Matriz de correlación para RFM
plt.figure(figsize=(8, 6))
correlation_matrix_rfm = rfm_df.corr()
sns.heatmap(correlation_matrix_rfm, annot=True, cmap="coolwarm", fmt=".2f")
plt.title('Matriz de Correlación entre Variables RFM')
plt.show()

In [ ]:
from sklearn.cluster import KMeans
# Definir una lista de posibles valores de k para el método del codo
k_values = range(2, 8)

# Inicializar listas para almacenar las métricas
inertia_values = []
# Realizar clustering con diferentes valores de k y calcular las métricas
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_df)
    labels = kmeans.labels_
    # Calcular la inercia
    inertia_values.append(kmeans.inertia_)

# Graficar el método del codo utilizando la inercia
plt.plot(k_values, inertia_values, 'bo-')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inercia')
plt.title('Método del Codo')
plt.show()

In [ ]:
# Aplicar K-Means con 4 clusters en el conjunto completo de datos
kmeans_4 = KMeans(n_clusters=4, random_state=42, n_init=10, max_iter=300)
rfm_df['Cluster'] = kmeans_4.fit_predict(rfm_df)

# Describir los clusters resultantes
cluster_summary = rfm_df.groupby('Cluster').agg({
    'RECENCIA': ['mean', 'median'],
    'FRECUENCIA': ['mean', 'median'],
    'TOTAL_ITEMS': ['mean', 'median'],
    'ACCOUNT_ID': 'count'
}).reset_index()

# Mostrar el resumen de los clusters
cluster_summary

In [ ]:
from sklearn.decomposition import PCA

# Asignar el valor óptimo de k (elegido por el método del codo)
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(rfm_df)

# Obtener las etiquetas de los clusters
labels = kmeans.labels_

# Añadir las etiquetas de los clusters al DataFrame
rfm_df['clusters'] = labels

# Reducir la dimensionalidad a 2 componentes principales con PCA para graficar
pca = PCA(n_components=2)
rfm_pca = pca.fit_transform(rfm_df.drop(columns=['clusters']))

# Graficar los clusters en el espacio reducido por PCA
plt.figure(figsize=(8, 6))
plt.scatter(rfm_pca[:, 0], rfm_pca[:, 1], c=labels, cmap='viridis')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title(f'Clusters K-means (k={optimal_k})')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Suponiendo que X es tu conjunto de datos y labels son las etiquetas del clustering
silhouette_scores = silhouette_score(rfm_df, labels)
davies_bouldin_scores = davies_bouldin_score(rfm_df, labels)

print(f'Silhouette Score: {silhouette_scores:.4f}')
print(f'Davies-Bouldin Score: {davies_bouldin_scores:.4f}')

In [ ]:
!pip install scikit-fuzzy
import skfuzzy as fuzz

In [78]:
# Definir una lista de posibles valores de m
m_values = np.arange(1.2, 3.8, 0.1)

# Inicializar listas para almacenar las métricas
silhouette_scores_fuzzy = []
davies_bouldin_scores_fuzzy = []

# Realizar clustering fuzzy con diferentes valores de m y calcular las métricas
for m in m_values:
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(rfm_df.T, 4, m, error=0.005, maxiter=1000)
    labels = np.argmax(u, axis=0)

    # Calcular la puntuación de silueta
    silhouette_scores_fuzzy.append(silhouette_score(rfm_df, labels))

    # Calcular el índice de Davies-Bouldin
    dunn_index = davies_bouldin_score(rfm_df, labels)
    davies_bouldin_scores_fuzzy.append(dunn_index)

# Encontrar el valor óptimo de m basado en la puntuación de silueta
optimal_index = np.argmax(silhouette_scores_fuzzy)
optimal_m = m_values[optimal_index]

In [ ]:
# Aplicar Fuzzy C-Means con el valor óptimo de m
n_clusters = 4
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(rfm_df.T, n_clusters, optimal_m, error=0.005, maxiter=1000)

# Obtener la pertenencia de cada punto al cluster más cercano
cluster_membership = np.argmax(u, axis=0)

# Añadir la asignación de clusters al DataFrame original
rfm_df['Fuzzy_Cluster'] = cluster_membership

# Resumir los clusters resultantes
fuzzy_cluster_summary = rfm_df.groupby('Fuzzy_Cluster').agg({
    'RECENCIA': ['mean', 'median'],
    'FRECUENCIA': ['mean', 'median'],
    'TOTAL_ITEMS': ['mean', 'median'],
    'ACCOUNT_ID': 'count'
}).reset_index()

# Imprimir el valor óptimo de m y el resumen de los clusters
print(f"Valor óptimo de m: {optimal_m}")
print()
print(fuzzy_cluster_summary)

In [ ]:
# Graficar los resultados en 2D
plt.figure(figsize=(10, 4))

# Resultados del Clustering K-means
plt.subplot(121)
plt.scatter(rfm_pca[:, 0], rfm_pca[:, 1], c=labels, cmap='viridis')
plt.title('Clustering K-means')

# Resultados del Clustering Fuzzy C-means
plt.subplot(122)
plt.scatter(rfm_pca[:, 0], rfm_pca[:, 1], c=cluster_membership, cmap='viridis')
plt.title('Clustering Fuzzy C-means')

plt.show()

In [ ]:
# Graficar la puntuación de silueta y el índice de Dunn
plt.figure(figsize=(8, 4))
plt.plot(m_values, silhouette_scores_fuzzy, 'bo-', label='Puntuación de silueta')
plt.plot(m_values, davies_bouldin_scores_fuzzy, 'ro-', label='Índice de Dunn')
plt.xlabel('Valor de m')
plt.ylabel('Métrica')
plt.title('Evaluación de Clustering Fuzzy C-means')
plt.legend()
plt.show()

In [ ]:
# Extraer los valores numéricos de las listas en Clustering Fuzzy
silhouette_scores_fuzzy_value = silhouette_scores_fuzzy[0] if isinstance(silhouette_scores_fuzzy, list) else silhouette_scores_fuzzy
davies_bouldin_scores_fuzzy_value = davies_bouldin_scores_fuzzy[0] if isinstance(davies_bouldin_scores_fuzzy, list) else davies_bouldin_scores_fuzzy

# Crear un DataFrame para comparar las métricas
comparison_df = pd.DataFrame({
    'Métrica': ['Silhouette Score', 'Davies-Bouldin Score'],
    'Clustering K-means': [silhouette_scores, davies_bouldin_scores],
    'Clustering Fuzzy': [silhouette_scores_fuzzy_value, davies_bouldin_scores_fuzzy_value]
})

# Mostrar el DataFrame comparativo
print(comparison_df)

# Visualización de la comparación
comparison_df.set_index('Métrica').plot(kind='bar', figsize=(10, 6), color=['skyblue', 'lightgreen'])
plt.title('Comparación de Métricas: Clustering K-means vs Clustering Fuzzy')
plt.ylabel('Valor de la Métrica')
plt.xticks(rotation=0)
plt.legend(loc='best')
plt.show()

__Resumen de las clasificaciones__:

* `Clúster 0`: Compradores Ocasionales Activos – compran regularmente, pero en cantidades moderadas.

* `Clúster 1`: Compradores Fieles y Activos – clientes que compran grandes volúmenes y con frecuencia, los mejores clientes.

* `Clúster 2`: Compradores Frecuentes y Moderados – compran de manera constante, pero en cantidades moderadas.

* `Clúster 3`: Compradores de Volumen Pequeño – clientes que hacen compras pequeñas y frecuentes.